In [ ]:
################################################################################################
################################################################################################
## REVISIÓN DE VERSIONES                                                                      ##
##                                                                                            ##
## v.2.0                                                                                      ##
## - scraping de una página específica de partidos diarios                                    ##
## - crea dataframe y lo envía a archivo .csv                                                 ##
##                                                                                            ##
################################################################################################
################################################################################################

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# def obtener_analisis(soup):
#     list_analisis = []
    
#     # ahora vamos a buscar "analisis" en toda la lista de matches, por lo tanto tenemos que poner de nuevo el código de búsqueda
#     r_analisis = requests.get(soup)
#     s_analisis = BeautifulSoup(r_analisis.text, 'lxml')
#     find_analisis = s_analisis.find('div', attrs={'class':'menu-scroll'}).select('a[href*="analisis"]')
#     analisis = find_analisis[0]
#     if analisis:
#         list_analisis.append(analisis.get('href'))
        
#     return list_analisis    

In [3]:
def obtener_stats(func_stats):
    stats_dict = {}
    
    date = func_stats.find('div', attrs={'class':'info match-link'})
    if date:
        stats_dict['date'] = date.get('starttime').replace('T',' ').replace(':00+00:00','')
    else:
        stats_dict['date'] = '-'
 
    draw = func_stats.find('div', attrs={'class':'content-box draw'}).find('div', attrs={'class':'cont'})
    if draw:
        stats_dict['draw'] = draw.get_text('/',strip=True).replace('%','')[4:10]
    else:
        stats_dict['draw'] = '-' 
        
    homeRank = func_stats.find('div', attrs={'class':'position t1'})
    if homeRank:
        stats_dict['homeRank'] = homeRank.get_text()
    else:
        stats_dict['homeRank'] = '-'    
            
    homeTeam = func_stats.find('p', attrs={'itemprop':'homeTeam'})
    if homeTeam:
        stats_dict['homeTeam'] = homeTeam.get_text('/', strip=True)
    else:
        stats_dict['homeTeam'] = '-'

    homeGoals = func_stats.find('div', attrs={'class':'content-box t-1'}).find('div', attrs={'class':'poss-box'})
    if homeGoals:
        stats_dict['homeGoals'] = homeGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')
    else:
        stats_dict['homeGoals'] = '-'     
        
    awayGoals = func_stats.find('div', attrs={'class':'content-box t-2'}).find('div', attrs={'class':'poss-box'})
    if awayGoals:
        stats_dict['awayGoals'] = awayGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')
    else:
        stats_dict['awayGoals'] = '-'     

    awayTeam = func_stats.find('p', attrs={'itemprop':'awayTeam'})
    if awayTeam:
        stats_dict['awayTeam'] = awayTeam.get_text('/', strip=True)
    else:
        stats_dict['awayTeam'] = '-'     
     
    awayRank = func_stats.find('div', attrs={'class':'position t2'})
    if awayRank:
        stats_dict['awayRank'] = awayRank.get_text()
    else:
        stats_dict['awayRank'] = '-' 
        
    competition = func_stats.find('h3', attrs={'class':'competition'})
    if competition:
        stats_dict['competition'] = competition.get_text('/', strip=True)
    else:
        stats_dict['competition'] = '-'
   
    return stats_dict

In [4]:
df = pd.DataFrame()

In [5]:
url = input('Introduce la url a scrapear: ')
#url = 'https://es.besoccer.com/livescore/2022-09-21'
besoccer = requests.get(url)

# por ejemplo: https://es.besoccer.com/livescore/2022-09-20

Introduce la url a scrapear: https://es.besoccer.com/livescore/2022-09-23


In [6]:
besoccer.status_code

200

In [7]:
s_besoccer = BeautifulSoup(besoccer.text, 'lxml')

In [8]:
matches = s_besoccer.find('div', attrs={'class':'matches'}).select('a[href*="/partido/"]')

In [9]:
[match.text for match in matches] 

['\n\n\n\nLiga C. J 5\n\n                ESPN Extra,Star+,SKY Sports,ESPN 4,UEFA TV            \n\n\nGeorgia\n\n\n\n18:00\n\n\n\nMacedonia del Norte\n\n23 sep 2022\n',
 '\n\n\n\nLiga D. J 5\n\n                Star+,SKY Sports,UEFA TV            \n\n\nEstonia\n\n\n\n18:00\n\n\n\nMalta\n\n23 sep 2022\n',
 '\n\n\n\nLiga A. J 5\n\n                ESPN,ESPN 3,Star+,SKY Sports,ESPN 4            \n\n\nItalia\n\n\n\n20:45\n\n\n\nInglaterra\n\n23 sep 2022\n',
 '\n\n\n\nLiga A. J 5\n\n                Star+,SKY Sports,UEFA TV,ZDF            \n\n\nAlemania\n\n\n\n20:45\n\n\n\nHungría\n\n23 sep 2022\n',
 '\n\n\n\nLiga B. J 5\n\n                Star+,SKY Sports,UEFA TV            \n\n\nFinlandia\n\n\n\n20:45\n\n\n\nRumanía\n\n23 sep 2022\n',
 '\n\n\n\nLiga B. J 5\n\n                Star+,SKY Sports,UEFA TV            \n\n\nBosnia\n\n\n\n20:45\n\n\n\nMontenegro\n\n23 sep 2022\n',
 '\n\n\n\nLiga C. J 5\n\n                Star+,SKY Sports,UEFA TV            \n\n\nBulgaria\n\n\n\n20:45\n\n\n\nGibraltar\

In [10]:
try:
    match = matches[0]
    match.get('href')
except Exception as e:
    print('No hay más partidos que analizar')
        

In [11]:
links_matches = [match.get('href') for match in matches]

In [12]:
for i in range(len(links_matches)):
    list_analisis = []
    r_match = requests.get(links_matches[i])
    s_match = BeautifulSoup(r_match.text, 'lxml')
    find_analisis = s_match.find('div', attrs={'class':'menu-scroll'}).select('a[href*="analisis"]')
    try:
        analisis = find_analisis[0]
        link_analisis = analisis.get('href')
        r_link_analisis = requests.get(link_analisis)
        s_link_analisis = BeautifulSoup(r_link_analisis.text, 'lxml')
        df = df.append(obtener_stats(s_link_analisis), ignore_index = True)
        print(i, link_analisis)
    except Exception as e:
        print(i,'.- La url ',links_matches[i],' no tiene sección análisis')
        print(i,'.-',e)

C:\Users\jesus\AppData\Local\Temp\ipykernel_9860\483865442.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


0 https://es.besoccer.com/partido/seleccion-georgia/seleccion-macedonia-fyr/2023104/analisis


C:\Users\jesus\AppData\Local\Temp\ipykernel_9860\483865442.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


1 https://es.besoccer.com/partido/seleccion-estonia/Malta-eu/2023155/analisis


C:\Users\jesus\AppData\Local\Temp\ipykernel_9860\483865442.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


...

C:\Users\jesus\AppData\Local\Temp\ipykernel_9860\483865442.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


47 https://es.besoccer.com/partido/qingdao-youth-island/jiangxi-liansheng/2022541932/analisis


C:\Users\jesus\AppData\Local\Temp\ipykernel_9860\483865442.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


48 https://es.besoccer.com/partido/cholet/concarneau/2023143610/analisis
49 https://es.besoccer.com/partido/fc-bastia-borgo/mans-uc-72/2023143608/analisis


C:\Users\jesus\AppData\Local\Temp\ipykernel_9860\483865442.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(obtener_stats(s_link_analisis), ignore_index = True)


In [13]:
df.style.set_properties(**{'text-align': 'left'}) 

,date,draw,homeRank,homeTeam,homeGoals,awayGoals,awayTeam,awayRank,competition
0,2022-09-23 16:00,2.8,4º,Georgia,33.2% - 1.64,44.9% - 1.93,Macedonia del Norte,6º,"Liga de las Naciones de la UEFA. Liga C, Jornada 5"
1,2022-09-23 16:00,3.1,4º,Estonia,34.5% - 1.62,43.2% - 1.84,Malta,3º,"Liga de las Naciones de la UEFA. Liga D, Jornada 5"
2,2022-09-23 18:45,5.9,9º,Italia,16.3% - 0.88,62.4% - 1.95,Inglaterra,14º,"Liga de las Naciones de la UEFA. Liga A, Jornada 5"
3,2022-09-23 18:45,0.2,8º,Alemania,87% - 4.73,5.8% - 1.49,Hungría,5º,"Liga de las Naciones de la UEFA. Liga A, Jornada 5"
4,2022-09-23 18:45,17.9,9º,Finlandia,31.4% - 0.83,34.8% - 0.89,Rumanía,10º,"Liga de las Naciones de la UEFA. Liga B, Jornada 5"
5,2022-09-23 18:45,13.7,2º,Bosnia,37.6% - 1.05,31.5% - 0.93,Montenegro,4º,"Liga de las Naciones de la UEFA. Liga B, Jornada 5"
6,2022-09-23 16:00,9.6,-,Croacia Sub 21,49.8% - 1.45,23.4% - 0.9,Dinamarca Sub 21,-,Clasificación Europeo Sub 21. Final
7,2022-09-23 16:00,1,-,Islandia Sub 21,79% - 3.39,9.3% - 1.19,República Checa Sub 21,-,Clasificación Europeo Sub 21. Final
8,2022-09-23 16:00,1.4,-,Eslovaquia Sub 21,65.6% - 2.8,17.8% - 1.45,Ucrania Sub 21,-,Clasificación Europeo Sub 21. Final
9,2022-09-23 18:00,5.1,-,Irlanda Sub 21,50.4% - 1.76,26.2% - 1.21,Israel Sub 21,-,Clasificación Europeo Sub 21. Final


In [14]:
df.to_csv('matches.csv', header = False)